This code pulls variables from multiple clean and processed csvs to create the dataframe used for the ordinal model of the first clips of each observation.

In [15]:
# import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()

'/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement'

In [16]:
# read in data
meta = pd.read_csv('clean-data/metadata.csv')
first_clips = pd.read_csv('clean-data/first_clips_clean.csv')
flight = pd.read_csv('processed-data/drone_variables_flight.csv')
clip = pd.read_csv('processed-data/drone_variables_clip2.csv')
env = pd.read_csv('processed-data/env_variables_clip2.csv')
ground = pd.read_csv('clean-data/ground_conditions.csv')

In [17]:
# Create dataframe for analysis of first clips
# Gather all columns in one dataframe
df1 = pd.merge(meta, first_clips, on = ['observation', 'site', 'flight'], how = 'outer')
df1 = pd.merge(df1, flight, on = 'flight', how = 'outer')
df1 = pd.merge(df1, clip, on = 'flight', how = 'outer')
df1 = pd.merge(df1, env, on = 'flight', how = 'outer')
df1 = pd.merge(df1, ground, on = 'flight', how = 'outer')
df1.replace('NAN', np.nan, inplace = True)

# add in flight type (initial or replacement)
df1['flight_type'] = ['initial' if '-01' in i else 'replacement' for i in df1['flight']]

# determine whether group is downwind or not
df1['wind_direction'] = pd.to_numeric(df1['wind_direction'])
df1['angle'] = abs(df1['wind_direction'] - df1['launch-group_bear'])
df1['wind_angle'] = [i if i < 180 else 360-i for i in df1['angle']]
df1['group_downwind'] = [True if i <45 else False for i in df1['wind_angle']]

# make boolean variable indicating whether other species are present
df1['other_spp'] = [True if i>t else False for i,t in zip(df1['group_size_total'],df1['group_size_spp'])]

# save dataframe
cols_to_keep = ['observation', 'site', 'date', 'flight', 'species',
       'clip_type', 'group_size_spp', 'group_size_total', 'young_present', 'group_behavior',
       'launch_alt', 'group_lat', 'group_lon', 'obs_ascent', 'launch_dist', 'approach_speed_mean', 'approach_speed_max',
       'approach_duration_minutes', 'approach_tortuousity', 'exchange_duration', 'takeoff_time', 'landing_time', 'clip_start_ascent', 'prox_move_v',
       'prox_move_h', 'bushiness_100m', 'max_windspeed', 'mean_windspeed', 'temperature_c',
       'wind_speed_ms', 'cloud_cover_%', 'humidity_%', 'group_downwind', 'clip_start_ascent_adj', 'other_spp']
df1.drop(df1.columns.difference(cols_to_keep), axis = 1, inplace = True)
df1.to_csv('/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/data/first_clips.csv', index = False)